In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim

import numpy as np

In [10]:
# UNIPoint model
#-------------------------------------------------------
# dimensions be like:
#-------------------------------------------------------
# input -> hidden state -> parameters -> basis functions -> intensity function (output)
#-------------------------------------------------------
# (batch_size, seq_len, n_features) -> [for iteration we take input] (batch_size, 1, n_features) ->
# -> (batch_size, 1, 1) -> (batch_size, 1, n_parameters * n_basis_functions) -> 
# -> (batch_size, 1, n_basis_functions) -> (batch_size, 1, 1) 
#-------------------------------------------------------


class UNIPoint(nn.Module):
    def __init__(self, batch_size, seq_len, n_features, n_parameters, n_basis_functions):
      """
      Input parameters:
      n_neurons - number of neurons inside RNN
      n_parameters - expecteed number of parameters in basis function
      n_basis_functions - number of basis functions
      """
      super(UNIPoint, self).__init__()

      self.rnn = nn.RNNCell(n_features, 1)
      self.hx = torch.randn(batch_size, 1) # initialize hidden state 
      self.h2p = nn.Linear(1, n_parameters * n_basis_functions)
      self.basis_res = torch.randn(batch_size, n_basis_functions) #initialize matrix for basis f-s calculations results
      self.Softplus = torch.nn.Softplus(beta = 1)
      self.n_basis_functions = n_basis_functions

    def ReLU(self, parameter_1, parameter_2, time):
      """Function to apply Rectified Linear Unit (ReLU) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
      self.output = torch.relu(self.parameters[:,parameter_1] * time + self.parameters[:,parameter_2] ) 
      return self.output
    
    def PowerLaw(self, parameter_1, parameter_2, time): # need to fix (see ReLU parameters and do the same)
      """Function to apply Power Law (PL) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
      self.output = self.parameters[:,parameter_1] * (1 + time)**( - self.parameters[:,parameter_2])
      return self.output


    def forward(self, X, time):
      """Input parameters:
          X - batch with data 
          time - column-vector with interarrival time in temporal point process (TPP)
      """
        
      output = []

      print("------------Learning process starts here------------")
      print()
      # for each time step (here X shape is (batch_size, seq_len, n_features) )
      for i in range(X.shape[1]):

          #print(X[:,i,:].shape, 'is X shape')
          #print(self.hx.shape, 'is self.hx shape')
          self.hx = self.rnn(X[:,i,:], self.hx)
          self.parameters = self.h2p(self.hx)
          
          for function in range(self.n_basis_functions): 
              # calculating numbers of parameters to take for basis function
              par1 = 2 * function
              par2 = 2 * function + 1
              
              #print("'function' iterator value is ", function)
              #print('par1, par2 are ', par1, par2)
              #print('X[:,i,:] shape is ', X[:,i,:].shape)
              #print()
              #print(self.basis_res[:, function].shape, 'is a shape of self.basis_res[:, function]')
              self.basis_res[:, function] = self.ReLU(par1, par2, X[:,i,1]) # here X[:,i,1] - tau
          
          self.sum_res = torch.sum(self.basis_res, 1)

          self.intensity_res = self.Softplus(self.sum_res)

          output.append(self.hx)
          
          print("Epoch", i+1, "out of", X.shape[1])
          print()

      print("------------Learning process is finished------------")

      return output, self.hx, self.parameters, self.basis_res, self.sum_res, self.intensity_res


In [11]:
# model evaluation

# X_batch dimension = (batch_size, seq_len, n_features)
X_batch = torch.tensor([[[1, 0.1],
                         [2, 0.2],
                         [3, 0.3],
                         [4, 0.4],
                         [5, 0.5],
                         [6, 0.6],
                         [7, 0.7]],
                        [[10, 1.0],
                         [8, 0.8], 
                         [6, 0.6],
                         [4, 0.4],
                         [2, 0.2],
                         [1, 0.1],
                         [2, 0.2]]], dtype = torch.float)

#print("shape of X_batch: ", X_batch.shape)

TIME = X_batch[:,:,1].detach().clone()
#for i in range(7):
#  print(X_batch[:,i,:])
#  print('this sequence shape is ', X_batch[:,i,:].shape)
#  print()

FIXED_BATCH_SIZE = 2 # our batch size is fixed for now
SEQ_LEN = 7
N_FEATURES = 2
#N_NEURONS = 1

N_PARAMETERS = 2
N_BASIS_FUNCTIONS = 4


model = UNIPoint(FIXED_BATCH_SIZE, SEQ_LEN, N_FEATURES, N_PARAMETERS, N_BASIS_FUNCTIONS)
print(model)
output_val, hid_states_val, params_value, basis_results, sum_results, res = model(X_batch, TIME)
print()
print('output_val')
print(output_val) # contains all output for all timesteps
print()
print('hid_states_val')
print(hid_states_val) # contains values for final state or final timestep, i.e., t=1
print()
print('params_value')
print(params_value) # contains values for parameters for final timesteps
print()
print('basis_results')
print(basis_results) # contains values of basis functions (now it is ReLU)
print()
print('sum_results')
print(sum_results) # contains values of basis functions
print()
print('res')
print(res) # contains values of SoftPlus activation

UNIPoint(
  (rnn): RNNCell(2, 1)
  (h2p): Linear(in_features=1, out_features=8, bias=True)
  (Softplus): Softplus(beta=1, threshold=20)
)
------------Learning process starts here------------

Epoch 1 out of 7

Epoch 2 out of 7

Epoch 3 out of 7

Epoch 4 out of 7

Epoch 5 out of 7

Epoch 6 out of 7

Epoch 7 out of 7

------------Learning process is finished------------

output_val
[tensor([[-0.9372],
        [-1.0000]], grad_fn=<TanhBackward>), tensor([[-0.9740],
        [-1.0000]], grad_fn=<TanhBackward>), tensor([[-0.9927],
        [-0.9999]], grad_fn=<TanhBackward>), tensor([[-0.9980],
        [-0.9980]], grad_fn=<TanhBackward>), tensor([[-0.9995],
        [-0.9716]], grad_fn=<TanhBackward>), tensor([[-0.9999],
        [-0.9004]], grad_fn=<TanhBackward>), tensor([[-1.0000],
        [-0.9753]], grad_fn=<TanhBackward>)]

hid_states_val
tensor([[-1.0000],
        [-0.9753]], grad_fn=<TanhBackward>)

params_value
tensor([[ 1.7341,  0.9731,  0.0634,  0.5576,  0.1505, -0.9563, -0.7113, -0.